In this notebook Longitude and latitude columns are added in order for further visualisations and processing. This proccess take a lot of time, therefore it is done in separate notebook and columns added before all further manipulation with the dataset.

In [63]:
!pip install requests geopy tqdm pandas numpy pyarrow


In [56]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

In [57]:
df = pd.read_parquet("../data/raw/pollution_dataset.parquet", engine="pyarrow")
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [ ]:
import pandas as pd
import numpy as np
import re

def clean_address(addr):
    if pd.isna(addr):
        return ""
    addr = addr.strip()
    addr = re.sub(r'\s+', ' ', addr)        # collapse spaces
    addr = re.sub(r'[^0-9A-Za-z, -]', '', addr)  # remove special chars
    addr = addr.replace(" ,", ",")
    return addr

df["CleanAddress"] = df["Address"].fillna("").apply(clean_address)


In [ ]:
site_keys = df[["State Code", "County Code", "Site Num", "CleanAddress"]].drop_duplicates()
site_keys = site_keys.reset_index(drop=True)

print("Unique site count:", len(site_keys))


In [ ]:
import requests

EPA_USER = "YOUR_EMAIL"
EPA_KEY  = "YOUR_KEY"

def get_monitor_coordinates(state_code, county_code, site_num):
    url = "https://aqs.epa.gov/data/api/monitors/bySite"
    params = {
        "email": EPA_USER,
        "key": EPA_KEY,
        "state": str(state_code).zfill(2),
        "county": str(county_code).zfill(3),
        "site": str(site_num).zfill(4),
    }

    try:
        r = requests.get(url, params=params).json()
        if "Data" in r and len(r["Data"]) > 0:
            d = r["Data"][0]
            return d.get("latitude"), d.get("longitude")
        else:
            return None, None
    except:
        return None, None


In [ ]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="usa_aqi_mapper")

def fallback_geocode(clean_addr):
    try:
        loc = geolocator.geocode(clean_addr + ", USA")
        if loc:
            return loc.latitude, loc.longitude
    except:
        pass
    return None, None


In [ ]:
from tqdm import tqdm

latitudes = []
longitudes = []

for _, row in tqdm(site_keys.iterrows(), total=len(site_keys)):
    lat, lon = get_monitor_coordinates(
        row["State Code"], row["County Code"], row["Site Num"]
    )

    # If EPA returns nothing → try fallback
    if lat is None or lon is None:
        lat, lon = fallback_geocode(row["CleanAddress"])
        time.sleep(1)  # be polite

    latitudes.append(lat)
    longitudes.append(lon)

site_keys["Latitude"] = latitudes
site_keys["Longitude"] = longitudes


In [ ]:
site_keys.to_parquet("epa_site_coordinates.parquet", index=False)
print("Saved → epa_site_coordinates.parquet")


In [ ]:
df = df.merge(
    site_keys,
    on=["State Code", "County Code", "Site Num"],
    how="left"
)
df.to_parquet("../data/processed/pollution_dataset_with_coordinates.parquet", index=False)
print("Saved → pollution_dataset_with_coordinates.parquet")

In [ ]:
"""
df = pd.merge(df, unique_addresses, on="Address", how="left")
df.to_parquet("../data/preprocessed/pollution_dataset_enriched.parquet", index=False)
"""